# IPFS Concepts

## IPFS Service Node

A node that is running the IPFS daemon and is connected to the IPFS network. It is able to provide content to other nodes and to retrieve content from other nodes.

In [1]:
# docker compose -f "tests/ipfs/docker-compose.yml" up -d --build

### Default Server Address

In [2]:
__IPFS_DEFAULT_URL__ = '/ip4/127.0.0.1/tcp/5001'

## IPFS-Toolkit

In [3]:
# pip install IPFS-Toolkit

### Connect

In [4]:
import ipfs_api

# Connect to the IPFS daemon
def connect():
    client = ipfs_api.ipfshttpclient
    return client.connect(__IPFS_DEFAULT_URL__)

#### Check Connection

In [5]:
def print_status():
    ipfs = connect()
    print('Client ID: ', ipfs.id())
    print('Client Version: ', ipfs.version())

print_status()

Client ID:  <ipfshttpclient2.client.base.ResponseBase: {'ID': '12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', 'PublicKey': 'CAESILveCqhrvYi7xfZkCFoytPVThX++ztHAh85jBT3xQ0Et', 'Addresses': ['/ip4/108.61.182.184/tcp/4001/p2p/12D3KooWPuZGkjnjLso3sLgNm5qaQMgiX77ndQ14JHqR2TwWp3Be/p2p-circuit/p2p/12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', '/ip4/108.61.182.184/udp/4001/quic-v1/p2p/12D3KooWPuZGkjnjLso3sLgNm5qaQMgiX77ndQ14JHqR2TwWp3Be/p2p-circuit/p2p/12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', '/ip4/108.61.182.184/udp/4001/quic/p2p/12D3KooWPuZGkjnjLso3sLgNm5qaQMgiX77ndQ14JHqR2TwWp3Be/p2p-circuit/p2p/12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', '/ip4/127.0.0.1/tcp/4001/p2p/12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', '/ip4/127.0.0.1/udp/4001/quic-v1/p2p/12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', '/ip4/127.0.0.1/udp/4001/quic-v1/webtransport/certhash/uEiAsG_OZi39bk9tp4XO6luYhWxo_vmHB30yD1HFNsCV1hg/certhash/uEiBU9zqLAAbnHBe6T6br9Bg29WL

### Publish

#### Strings

In [6]:
def add_str(string: str):
    with connect() as c:
        return c.add_str(string)

returned_hash = add_str(u'Hello World!')
print(returned_hash)

Qmf1rtki74jvYmGeqaaV51hzeiaa6DyWc98fzDiuPatzyy


#### Bytes

In [7]:
def add_bytes(data: bytes):
    with connect() as c:
        return c.add_bytes(data)
    
returned_hash = add_bytes(b'Hello World!')
print(returned_hash)

Qmf1rtki74jvYmGeqaaV51hzeiaa6DyWc98fzDiuPatzyy


#### JSON

In [8]:
def add_json(json: dict):
    with connect() as c:
        return c.add_json(json)
    
returned_hash = add_json({'hello': 'world'})
print(returned_hash)

QmNrEidQrAbxx3FzxNt9E6qjEDZrtvzxUVh47BXm55Zuen


#### Files

In [9]:
def add_file(path: str):
    with connect() as c:
        return c.add(path)
    
returned_hash = add_file('concepts_cid_test.json')
print(returned_hash['Hash'])

QmUJarh6GuMaNgpkvkC1CoRk8qDk6J39CopoYi9fVdSEiv


## Hash calculation

In [10]:
# pip install pycroptodome
# pip install py-cid
# pip install merkly

In [11]:
import io
import base58
# import BytesIO
from typing import Callable, Iterable
from cid import CIDv1, make_cid, CIDv0
# from multiformats_cid import make_cid, CIDv0, CIDv1, cid
# from Crypto.Hash import SHA256
import hashlib
from merkly.mtree import MerkleTree
import multihash
import codecs

def as_chunks(stream: io.BytesIO, chunk_size: int) -> Iterable[bytes]:
    while len(chunk := stream.read(chunk_size)) > 0:
        yield chunk

def chunk_to_leaf(chunk: bytes) -> str:
    return hashlib.sha256(chunk).hexdigest()

def data_to_tree(data: bytes, chunk_size: int) -> list[str]:
    # Create a buffer to hold the tree
    tree: list[bytes] = []

    # Iterate over the chunks
    for chunk in as_chunks(io.BytesIO(data), chunk_size):
        # Calculate the leaf hash
        tree.append(chunk_to_leaf(chunk))
        # print(tree)

    # Return the tree
    return tree

def tree_to_root(tree: list[str]) -> str:
    sha256_hash_funciton: Callable[[str], str] = lambda x, y: str(hashlib.sha256(x.encode() + y.encode()).hexdigest())

    # Calculate the root hash
    mt = MerkleTree(tree, sha256_hash_funciton)
    root = mt.root
    print(f'raw_root: {root}')
    # root = codecs.decode(root, 'hex')
    # print('root:', root.hex())
    return root

def calculate_ipfs_hash(data: bytes):
    # Calculate the SHA256 hash of the data
    sha256_hash = hashlib.sha256(data).digest()

    # Create a multihash for the SHA256 hash
    # Multihash format: <hash function code><digest size><digest>
    # SHA2-256 function code is 0x12, digest size is 0x20 (32 in decimal)
    mh = b'\x12\x20' + sha256_hash

    # Create a CIDv0
    # cidv0: CIDv0 = CIDv0(mh)

    # return cidv0

    # Create a CIDv0 for the multihash
    c_old: CIDv0 = make_cid(base58.b58encode(mh))
    print('CIDv0:', c_old)
    print(f'CIDv0 to CIDv1: {c_old.to_v1()}')

    c: CIDv1 = make_cid(1, 'dag-pb', mh)
    # c.buffer = c.encode()

    return str(c)

def calculate_cid_from_tree(data: bytes, chunk_size: int = 4) -> str:
    # Calculate the tree
    tree = data_to_tree(data, chunk_size)

    # Calculate the root hash
    root = tree_to_root(tree)
    # print('root:', root)
    # root_hashed_256 = hashlib.sha256(root.encode()).digest()
    root_hashed_256 = codecs.decode(root, 'hex')
    print(f'root_hashed_256: {root_hashed_256.hex()}')

    mh = multihash.encode(root_hashed_256, 'sha2-256')
    print('mh:', mh)

    # Create a CIDv0 for the root hash
    # mh_root = b'\x12\x20' + codecs.decode(root, 'hex')
    # mh_root = b'\x12\x20' + codecs.decode(root, 'hex')
    mh_root = b'\x12\x20' + root_hashed_256
    print('mh_root:', mh_root)
    c_old: CIDv0 = make_cid(base58.b58encode(mh_root))
    print('CIDv0:', c_old)

    # Create a CIDv1 for the root hash
    cidv0_2: CIDv0= make_cid(0, 'dag-pb', mh_root)
    print('CIDv0_2:', cidv0_2)

    # Return the CIDv0
    return str(cidv0_2)

# Read the file data
with open('concepts_cid_test.json', 'rb') as f:
    data = f.read()

# Calculate the IPFS hash
# ipfs_hash = calculate_ipfs_hash(data)
print(f'hash from mt: {calculate_cid_from_tree(data)}')
print(f'hash from ipfs add function: {returned_hash["Hash"]}')

# print('IPFS Hash:', ipfs_hash)

raw_root: fe9b05ddd2c30b4134f8b5e1b9ecf8b9f8485b8090565d006ec009b9c752f308
root_hashed_256: cea396b4dfdde67e897d80381a05ad3b401a33f7caaaa1b6e53ec9b761f796e3
mh: b'\x12 \xce\xa3\x96\xb4\xdf\xdd\xe6~\x89}\x808\x1a\x05\xad;@\x1a3\xf7\xca\xaa\xa1\xb6\xe5>\xc9\xb7a\xf7\x96\xe3'
mh_root: b'\x12 \xce\xa3\x96\xb4\xdf\xdd\xe6~\x89}\x808\x1a\x05\xad;@\x1a3\xf7\xca\xaa\xa1\xb6\xe5>\xc9\xb7a\xf7\x96\xe3'
CIDv0: QmcFHNeDmYENvuqQ5hRVzR3rArXDh86BjyWVJ7mKzrWhsx
CIDv0_2: QmcFHNeDmYENvuqQ5hRVzR3rArXDh86BjyWVJ7mKzrWhsx
hash from mt: QmcFHNeDmYENvuqQ5hRVzR3rArXDh86BjyWVJ7mKzrWhsx
hash from ipfs add function: QmUJarh6GuMaNgpkvkC1CoRk8qDk6J39CopoYi9fVdSEiv


### IPLD

In [12]:
# pip install ipld - Not working on Python 3.12.0

In [13]:
# from ipld import Ipld, Link

# # Create a node
# node = Ipld()

# # Add data to the node
# node['data'] = b'Hello, world!'

# # Create a link to another node
# link = Link('/ipfs/QmT78zSuBmuS4z925WZfrqQ1qHaJ56DQaTfyMUF7F8ff5o')
# node['link'] = link

# # Serialize the node
# serialized_node = node.serialize()

# # Calculate the CID of the node
# cid = node.cid()

# print('CID:', cid)